# Extração de grafos de conhecimento de bulas de medicamentos de arquivos PDF extraídos do bulário eletrônico da ANVISA


**Materiais úteis**
- [Preparing PDFs for RAGs: um comparativo entre bibliotecas](https://towardsdatascience.com/preparing-pdfs-for-rags-b1579fc697f1/)
- [Build a knowledge graph from documents using Docling](https://alain-airom.medium.com/build-a-knowledge-graph-from-documents-using-docling-8bc05e1389f7)
- [Using PyMuPDF4LLM: A Practical Guide for PDF Extraction in LLM & RAG Environments](https://medium.com/@danushidk507/using-pymupdf4llm-a-practical-guide-for-pdf-extraction-in-llm-rag-environments-63649915abbf)
- [How To Extract Structured Knowledge From Unstructured PDFs?](https://medium.com/@dreamai/how-to-extract-structured-knowledge-from-unstructured-pdfs-b5b1b101041e)
- [GraphRAG Python Package: Accelerating GenAI With Knowledge Graphs](https://neo4j.com/blog/news/graphrag-python-package/)
- [How to parse PDF docs for RAG](https://cookbook.openai.com/examples/parse_pdf_docs_for_rag)
- [Build Knowledge Graph RAG with LlamaIndex from PDF Document](https://www.linkedin.com/pulse/build-knowledge-graph-rag-llamaindex-from-pdf-documents-peng-wang-areoc/)
- [Knowledge Graph from Unstructured Text](https://www.superteams.ai/blog/knowledge-graph-from-unstructured-text)

In [4]:
!python3 -m spacy download pt_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 MB 21.2 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_md')


In [ ]:
from src.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from src.data_processing.pdf_and_markdown_pipeline import PdfAndMarkdownPipeline
from src.data_processing.chunks import HEADERS_TO_SPLIT_ON

PDF_FILE_PATH = f"{RAW_DATA_DIR}/bula_1748876242121.pdf"
MD_FILE_PATH = f"{PROCESSED_DATA_DIR}/bula_1748876242121.pdf.md"

In [ ]:
pdf_md_pipeline = PdfAndMarkdownPipeline()
pdf_md_pipeline.pdf_to_markdown(
    source_file_path=PDF_FILE_PATH,
    dest_dir_path=PROCESSED_DATA_DIR
)

2025-06-04 14:36:36.029 | INFO     | src.config:<module>:11 - PROJ_ROOT path is: /home/marilia/git/pos/tcc_ufg_akcit_nlp
/home/marilia/git/pos/tcc_ufg_akcit_nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for StandardPdfPipeline with options hash 41fbf895b49bedfbab01cd9782e93016
INFO:docling.models.factories.base_factory:Loading plugin 'docling_defaults'
INFO:docling.models.factories:Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.utils.accelerator_utils:Accelerator device: 'cpu'
INFO:docling.models.fac

PdfAndMarkdownPipeline(md_documents=None, chunk_documents=[], chunk_object=None)

In [2]:
pdf_md_pipeline.load_markdown(
    source_file_path=MD_FILE_PATH
)
pdf_md_pipeline.md_documents

INFO:src.data_processing.pdf_to_markdown:Document /home/marilia/git/pos/tcc_ufg_akcit_nlp/data/processed/bula_1748876242121.pdf.md loaded.


[Document(metadata={'source': '/home/marilia/git/pos/tcc_ufg_akcit_nlp/data/processed/bula_1748876242121.pdf.md'}, page_content="paracetamol\n\nAché Laboratórios Farmacêuticos S.A. C omprimidos revestidos 750 mg\n\nBULA PARA PACIENTE\n\nBula de acordo com a Resolução-RDC nº 47/2009\n\nI.IDENTIFICAÇÃO DO MEDICAMENTO\n\nparacetamol\n\nMedicamento Genérico Lei nº 9.787, de 1999\n\nAPRESENTAÇÕES\n\nComprimidos revestidos de 750 mg: embalagens com 20 ou 200 comprimidos.\n\nUSO ORAL\n\nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS\n\nCOMPOSIÇÃO\n\nCada comprimido revestido contém:\n\nparacetamol..........................................................................................................................................750 mg\n\nExcipientes: povidona, amidoglicolato de sódio, amido, ácido esteárico, álcool polivinílico, macrogol, talco e dióxido de titânio.\n\nII. INFORMAÇÕES AO PACIENTE\n\n1. PARA QUE ESTE MEDICAMENTO É INDICADO?\n\nEste medicamento é indicado para a redução da febre 

In [3]:
pdf_md_pipeline.get_chunks(
    chunk_size=400,
    chunk_overlap=100,
    headers_to_split_on=HEADERS_TO_SPLIT_ON,
)
pdf_md_pipeline.chunk_documents

["paracetamol\n\nAché Laboratórios Farmacêuticos S.A. C omprimidos revestidos 750 mg\n\nBULA PARA PACIENTE\n\nBula de acordo com a Resolução-RDC nº 47/2009\n\nI.IDENTIFICAÇÃO DO MEDICAMENTO\n\nparacetamol\n\nMedicamento Genérico Lei nº 9.787, de 1999\n\nAPRESENTAÇÕES\n\nComprimidos revestidos de 750 mg: embalagens com 20 ou 200 comprimidos.\n\nUSO ORAL\n\nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS\n\nCOMPOSIÇÃO\n\nCada comprimido revestido contém:\n\nparacetamol..........................................................................................................................................750 mg\n\nExcipientes: povidona, amidoglicolato de sódio, amido, ácido esteárico, álcool polivinílico, macrogol, talco e dióxido de titânio.\n\nII. INFORMAÇÕES AO PACIENTE\n\n1. PARA QUE ESTE MEDICAMENTO É INDICADO?\n\nEste medicamento é indicado para a redução da febre e para o alívio temporário de dores leves a moderadas, tais como dores associadas a resfriados comuns, dor de cabeça, dor no co

[Document(metadata={}, page_content='paracetamol  \nAché Laboratórios Farmacêuticos S.A. C omprimidos revestidos 750 mg  \nBULA PARA PACIENTE  \nBula de acordo com a Resolução-RDC nº 47/2009  \nI.IDENTIFICAÇÃO DO MEDICAMENTO  \nparacetamol  \nMedicamento Genérico Lei nº 9.787, de 1999  \nAPRESENTAÇÕES  \nComprimidos revestidos de 750 mg: embalagens com 20 ou 200 comprimidos.  \nUSO ORAL  \nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS'),
 Document(metadata={}, page_content='USO ORAL  \nUSO ADULTO E PEDIÁTRICO ACIMA DE 12 ANOS  \nCOMPOSIÇÃO  \nCada comprimido revestido contém:  \nparacetamol..........................................................................................................................................750 mg  \nExcipientes: povidona, amidoglicolato de sódio, amido, ácido esteárico, álcool polivinílico, macrogol, talco e dióxido de titânio.'),
 Document(metadata={}, page_content='II. INFORMAÇÕES AO PACIENTE  \n1. PARA QUE ESTE MEDICAMENTO É INDICADO?  \nEste medicamen